# Preparando os dados

In [1]:
import wntr
import folium
import numpy as np
import pandas as pd
from pyproj import Proj
from datetime import date
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
#Obtem os dados de diametro, tamanho e rugosidade do arquivo:
def Description_pipe(inp_file):
    network = wntr.network.WaterNetworkModel(inp_file)
    pipe_data = {}
    for link_name, link in network.links():
        if isinstance(link, wntr.network.elements.Pipe):
            length = link.length
            diameter = link.diameter
            roughness = link.roughness
            pipe_data[link_name] = {
                "Length (m)": length,
                "Diameter (m)": diameter #,
                # "Roughness (m)": roughness
            }
    pipe_table = pd.DataFrame(pipe_data).T
    return pipe_table

#Substituir valores da coluna material (kmeans permite apenas valores numericos)
def mapear_valores(valor):
    if valor == 'PVC':
        return 1
    elif valor == 'PVC DEFOFO':
        return 2
    elif valor == 'PEAD':
        return 3
    elif valor == 'FF':
        return 4
    else:
        return None
    
#  Realiza o processo de ETL inicial:
def etl_inp(pipe_data):
    pipe_data = pipe_data.reset_index()
    pipe_data.rename(columns={'index': 'ID'}, inplace=True) 
    pipe_data['ID'] = pipe_data['ID'].str.replace('P', '999')
    pipe_data['ID'] = pipe_data['ID'].astype(int)
    return pipe_data

# Função para unir os dois dataframes (informações do pipe + idade e material de cada pipe)
def merged(pipe_description_ETL,df_descricao_pipes):
    merged_df = pd.merge(pipe_description_ETL, df_descricao_pipes, on='ID', how='left')
    merged_df = merged_df.set_index('ID')
    merged_df.to_excel('C:/Users/kaiorodrigues/Desktop/WNTR/1_Data/Materiais/test.xlsx',index=False)
    # substitui os valores NAN por valores especificos
    merged_df['DATA_IMPLA'].fillna(value='1996-03-06', inplace=True)
    merged_df['MATERIAL'].fillna(value='PVC', inplace=True)
    # Converta a coluna 'Data de Nascimento' para datetime
    merged_df['DATA_IMPLA'] = pd.to_datetime(merged_df['DATA_IMPLA'])
    today = date.today()
    merged_df['Idade'] = today.year - merged_df['DATA_IMPLA'].dt.year
    merged_df['Idade'] = merged_df['Idade'].fillna(0).astype(int)
    merged_df['Material'] = merged_df['MATERIAL'].apply(mapear_valores)
    merged_df = merged_df.drop(['DATA_IMPLA', 'MATERIAL'], axis=1)
    return merged_df


In [3]:
# Arquivo Com a simulação
inp_file = "C:\\Users\\kaiorodrigues\\Desktop\\WNTR\\1_Data\\Vale_do_Amanhecer-Modelo_Final(Demanda_Nova).inp"

In [4]:
pipe_description = Description_pipe(inp_file)
pipe_description_ETL = etl_inp(pipe_description)
Description_material = pd.read_excel("C:/Users/kaiorodrigues/Desktop/WNTR/1_Data/Materiais/Rede_VAM.xlsx")
df_descricao_pipes = Description_material.loc[:, ['ID','DATA_IMPLA','MATERIAL']]
merged_df = merged(pipe_description_ETL,df_descricao_pipes)

c:\Users\kaiorodrigues\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [5]:
#  Removemos a linha com id 45 pq vem da capitação (pois importa apenas oque vai após o reservatório)
capitacao = 45
merged_df = merged_df.drop(capitacao)

# Utilizando o método Kmeans

In [6]:
def cluster(df,n_clusters):
    # criar um objeto KMeans e ajustá-lo ao df_descricao_pipes
    kmeans = KMeans(n_clusters=n_clusters).fit(df)
    # obter as etiquetas de cluster para cada registro
    labels = kmeans.labels_
    # obter os centróides de cada cluster
    centroids = kmeans.cluster_centers_
    # Acesse os rótulos dos clusters atribuídos a cada amostra
    rotulos = kmeans.labels_
    df['Grupo'] = labels
    return df

In [7]:
df = cluster(merged_df,5)
df = df.reset_index()
df['ID'] = df['ID'].astype(str).str.replace('999', 'P')
df = df.set_index('ID')

In [8]:
df

,Length (m),Diameter (m),Idade,Material,Grupo
ID,,,,,
0,0.812481,0.25,27,1,1
1,6.294194,0.30,27,2,1
10,226.210125,0.11,27,1,4
100,14.673712,0.06,27,1,1
101,49.187364,0.06,27,1,3
...,...,...,...,...,...
P43,8.220000,0.25,27,1,1
P45,9.280000,0.25,27,1,1
P47,10.100000,0.25,27,1,1


In [9]:
df.to_excel("C:\\Users\\kaiorodrigues\\Desktop\\WNTR\\cluster_Vale_do_amanhecer.xlsx")

In [10]:
contagem = df['Grupo'].value_counts()

# Exibir a contagem
print(contagem)


1    359
3    149
0     54
4     24
2      3
Name: Grupo, dtype: int64
